<a href="https://colab.research.google.com/github/julsanchezsa/MetNumUN2022I/blob/master/Lab_9/Week4IterativeMethodsForLinearSystems_Group6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
def jacobi(A, b, eps=1e-12, iter=50): 
    Diag_1 = np.diag(A)  
    B = -A.copy()

    np.fill_diagonal(B, 0) 
    invD = np.diag(1.0 / Diag_1)
    
    BB = invD @ B 
    c = invD @ b 
    
    x = np.ones(n)
    for i in range(n_iter): x = BB @ x + c

    return x 

In [ ]:
x = jacobi(A, b)
print(x)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]


In [ ]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(jacobi(A1, b)-np.linalg.solve(A1, b)))

0.38959181027260875 2.005305120107153e-17
0.4185783948614869 1.6502682545529e-17
0.4522284025473819 3.622208959480119e-17
0.4917667095178099 2.72080745804227e-17
0.5388887887486234 2.660694867584809e-17
0.5960110344093966 1.0255972154762448e-15
0.6667001660296402 2.770910834768342e-13
0.7564517359241753 1.4990529163071287e-10
0.8742017351588476 2.0161596093475135e-07
1.0355299928250665 0.0009191717405677889
1.2702850939751231 23.48163367953916
1.6439565658213244 8260242.793633645
2.334809111760855 261149802433164.78
4.080768845910033 1.3716691464130671e+26
30.715327603064885 1.739863582849129e+61


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
def seidel(A, b, eps=1e-17, iter=100):

    x = np.zeros_like(b, dtype=np.double)
    
    #Iteration
    for k in range(iter):
        
        x_old  = x.copy()
        
        #Loop over rows
        for i in range(A.shape[0]):
            x[i] = (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,(i+1):], x_old[(i+1):])) / A[i ,i]
            
        #Stop condition 
        if np.linalg.norm(x - x_old, ord=np.inf) / np.linalg.norm(x, ord=np.inf) < eps:
            break
            
    return x  
        

In [ ]:
x = seidel(A,b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [ ]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(seidel(A1, b)-np.linalg.solve(A1, b)))

0.38959181027260875 1.5540063044689707e-17
0.4185783948614869 2.303412968156487e-17
0.4522284025473819 3.3107778123195866e-17
0.4917667095178099 1.717836144195444e-17
0.5388887887486234 1.8703665918870363e-17
0.5960110344093966 2.6208106743381504e-17
0.6667001660296402 3.222105849667643e-17
0.7564517359241753 3.894444544739273e-17
0.8742017351588476 4.3610271956070115e-17
1.0355299928250665 7.521581756278068e-17
1.2702850939751231 6.691626947686432e-17
1.6439565658213244 1.0829177162645093e-16
2.334809111760855 1.576213700060856e-16
4.080768845910033 4.2966547712338835e-16
30.715327603064885 inf


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
def minres(A, b, n_iter=100):
    x = np.ones(b.shape[0])
    T = []
    for _ in range(n_iter):
        r = A @ x - b
        tau = (r @ A @ r)/np.linalg.norm(A @ r)**2
        x = x - tau*r
        T.append(tau)
    return x, T

x = minres(A, b)[0]

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [ ]:
minres(A, b)[1]

[0.049458091915919704,
 0.06632815899696641,
 0.056465283719342306,
 0.05913686144833426,
 0.056149365850670686,
 0.05962056630692138,
 0.05532125962259336,
 0.06019139542967885,
 0.054726519624928915,
 0.060918354718292295,
 0.05478879426160098,
 0.061543172821534486,
 0.055177211118735095,
 0.06162125672274042,
 0.055145295102092665,
 0.0617011494677153,
 0.05472584159792917,
 0.062121454652208655,
 0.054002133608905656,
 0.0644186170983185,
 0.05443303564328217,
 0.06011645923749506,
 0.06271150857694172,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0